In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import edhec_risk_kit as erk


In [2]:
def bt_mix(r1, r2, allocator, **kwargs):
    """
    Runs a backtest of allocatin between two sets of returns
    r1 and r2 are T x N dataframes or returns where T is the time step index and N is the number of scenarios
    allocator is a function that takes returns and parameters, and produces an allocation as a Tx1 dataframe
    Returns T x N DF of resulting scenarios
    """
    
    if not r1.shape == r2.shape:
        raise ValueError('r1 and r2 need to be the same shape')
    weights = allocator(r1, r2, **kwargs)
    if not weights.shape == r1.shape:
        raise ValueError('Allocator returned weights that dont match r1')
    r_mix = weights * r1 + (1-weights) * r2
    return r_mix

In [3]:
def fixedmix_allocator(r1, r2, w1, **kwargs):
    """
    Produces a time series over T steps of allocations between PSP and GHP across N scenarios
    PSP and GHP are TxN df of returns, each column is a scenario
    Returns a TXN dataframe of PSP weights
    """
    return pd.DataFrame(data=w1, index = r1.index, columns=r1.columns)

In [4]:
rates, zc_prices = erk.cir(10, 500, b=.03, r_0=.03)
price_10 = erk.bond_price(10, 100, .05, 12, rates)
price_30 = erk.bond_price(30, 100, .05, 12, rates)
rets_30 = erk.bond_total_return(price_30, 100, .05, 12)
rets_10 = erk.bond_total_return(price_10, 100, .05, 12)
rets_bonds = erk.bt_mix(rets_10, rets_30, allocator = erk.fixedmix_allocator, w1 = .6)
mean_rets_bonds = rets_bonds.mean(axis = 'columns')
erk.summary_stats(pd.DataFrame(mean_rets_bonds))

,Annualized Return,Annualized Vol,Skewness,Kurtosis,Cornish-Fisher VaR (5%),Historic CVaR (5%),Sharpe Ratio,Max Drawdown
0,0.036048,0.003402,-0.332262,3.26318,-0.001262,-0.000666,1.730295,0.0


In [5]:
price_eq = erk.gbm(n_years=10, n_scenarios=500, mu=.07, sigma=.15)
rets_eq = price_eq.pct_change().dropna()
rets_zc = zc_prices.pct_change().dropna()

In [6]:
rets_7030b = erk.bt_mix(rets_eq, rets_bonds, allocator=erk.fixedmix_allocator, w1 = .7)
rets_7030b_mean = rets_7030b.mean(axis = 1)
erk.summary_stats(pd.DataFrame(rets_7030b_mean))

,Annualized Return,Annualized Vol,Skewness,Kurtosis,Cornish-Fisher VaR (5%),Historic CVaR (5%),Sharpe Ratio,Max Drawdown
0,0.059518,0.00462,-0.060297,2.718735,-0.002616,-0.002088,6.218809,0.0


In [7]:
pd.concat([
    erk.terminal_stats(rets_bonds, name = 'FI'),
    erk.terminal_stats(rets_eq, name = 'EQ'),
    erk.terminal_stats(rets_7030b, name = '70/30')
], axis = 1)

,FI,EQ,70/30
mean,1.385976,1.965424,1.780055
std,0.108114,0.992514,0.617410
p_breach,NaN,0.036000,0.016000
e_short,NaN,0.170497,0.073954
p_reach,NaN,NaN,NaN
e_surplus,NaN,NaN,NaN


# Glide Paths for Allocation

In [8]:
def glidepath_allocator(r1, r2, start_glide = 1, end_glide = 0):
    """
    Simulates a TDF style move from r1 to r2
    """
    n_points = r1.shape[0]
    n_col = r1.shape[1]
    path = pd.Series(data = np.linspace(start_glide, end_glide, num = n_points))
    paths = pd.concat([path]*n_col, axis = 1)
    paths.index = r1.index
    paths.columns = r1.columns
    return paths

In [9]:
rets_g8020 = erk.bt_mix(rets_eq, rets_bonds, glidepath_allocator, start_glide = .8, end_glide = .2)

In [10]:
pd.concat([
    erk.terminal_stats(rets_bonds, name = 'FI'),
    erk.terminal_stats(rets_eq, name = 'EQ'),
    erk.terminal_stats(rets_7030b, name = '70/30'),
    erk.terminal_stats(rets_g8020, name = 'Glide 80/20')
], axis = 1)

,FI,EQ,70/30,Glide 80/20
mean,1.385976,1.965424,1.780055,1.663910
std,0.108114,0.992514,0.617410,0.438177
p_breach,NaN,0.036000,0.016000,0.006000
e_short,NaN,0.170497,0.073954,0.054025
p_reach,NaN,NaN,NaN,NaN
e_surplus,NaN,NaN,NaN,NaN


In [ ]:
# the glide path doesn't really solve the problem